# 데이터 준비하기: 다듬기, 변형, 병합

데이터 분석과 모델링 작업은 데이터를 불러오고, 다듬고, 변형하고 재정렬하는, 데이터 준비 과정에 많은 시간을 사용한다.

## 7.1. 데이터 합치기

pandas 객체에 저장된 데이터는 여러 내장 함수를 이용해서 합칠 수 있다.

* pandas.merge - sql 의 join 과 유사
* pandas.concat - 하나의 축에 따라 객체를 이어 붙인다
* combine_first - 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다

### 7.1.1. 데이터베이스 스타일로 DataFrame 합치기

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [2]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
               'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [3]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [4]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [5]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


merge 수행시 어떤 key 값을 기준으로 수행할지 지정하지 않았는데, 기본값은 같은 key 이름을 기준으로 수행한다.<br/>
위와 같은 예에서는 "key" 가 될 수 있다.

In [6]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


만약 동일한 key 이름을 갖고 있지 않다면, 지정해 주면 된다.

In [7]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [8]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [9]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


M:N 관계도 병합이 가능하나, 직관적이진 않다.

In [10]:
df1 = DataFrame({'key':['b', 'b', 'a', 'c' , 'a', 'b'],
                'data1': range(6)})

In [11]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                'data2':range(5)})

In [12]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [13]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


M:N 관계에서 조인은 두 로우의 데카르트 곱을 반환한다.<br/>
df1 에는 3개의 'b' 로우가 있고 오른쪽에는 2개의 'b' 로우가 있는데, 결과는 6개의 'b' 로우가 된다

In [14]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


여러 개의 키를 병합하려면 칼럼 이름이 들어간 리스트를 넘기면 된다.

In [15]:
left = DataFrame({'key1':['foo', 'foo', 'bar'],
                 'key2': ['one', 'two', 'own'],
                 'lval': [1, 2, 3]})

In [16]:
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                  'key2': ['one', 'one', 'one', 'two'],
                  'rval': [4, 5, 6, 7]})

In [17]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,own,3.0,NaN
4,bar,one,NaN,6.0
5,bar,two,NaN,7.0


두 DataFrame 에 대해 merge 연산을 수행할 때 동일한 이름의 컬럼이 있다면 어떻게 해야 할까?<br/>
merge 함수에는 suffixes 라는 인자를 지원한다

In [18]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,own,3,one,6
5,bar,own,3,two,7


In [19]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,own,3,one,6
5,bar,own,3,two,7


merge 함수 인자 목록

<table align="left">
    <thead>
        <td>인자</td>
        <td>설명</td>
    </thead>
    <tbody>
        <tr>
            <td>left</td>
            <td>merge 하려는 DataFrame 중 왼쪽에 위치한 DataFrame</td>
        </tr>
        <tr>
            <td>right</td>
            <td>merge 하려는 DataFrame 중 오른쪽에 위치한 DataFrame</td>
        </tr>
        <tr>
            <td>how</td>
            <td>조인방법. 'inner'(기본값), 'outer', 'left', 'right'</td>
        </tr>
        <tr>
            <td>on</td>
            <td>조인하려는 로우 이름, 반드시 두 DataFrame 객체 모두에 있는 이름이어야 한다.<br/>만약 명시되지 않고 다른 조인 키도 주어지지 않으면 left와 right에서 공통되는 칼럼을 조인 키로 사용한다.</td>
        </tr>
        <tr>
            <td>left_on</td>
            <td>조인 키로 사용할 left DataFrame 의 칼럼</td>
        </tr>
        <tr>
            <td>right_on</td>
            <td>조인 키로 사용할 right DataFrame 의 칼럼</td>
        </tr>
        <tr>
            <td>left_index</td>
            <td>조인 키로 사용할 left DataFrame 의 색인 로우(다중 색인일 경우의 키)</td>
        </tr>
        <tr>
            <td>right_index</td>
            <td>조인 키로 사용할 right DataFrame 의 색인 로우(다중 색인일 경우의 키)</td>
        </tr>
        <tr>
            <td>sort</td>
            <td>조인 키에 따라 병합된 데이터를 사전 순으로 정렬. 기본 값은 True</td>
        </tr>
        <tr>
            <td>suffixes</td>
            <td>칼럼 이름이 겹칠 경우 각 칼럼 이름 뒤에 붙일 문자열의 튜플. 기본 값은 ('_x', '_y')</td>
        </tr>
        <tr>
            <td>copy</td>
            <td>False 일 경우 예외적인 경우에 결과로 데이터가 복사되지 않도록 한다. 기본 값은 항상 복사가 이루어진다.</td>
        </tr>
    </tbody>
</table>

### 7.1.2. 색인 머지하기

In [20]:
left1 = DataFrame({'key' : ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})

In [21]:
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [22]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [23]:
right1

,group_val
a,3.5
b,7.0


In [24]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [25]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


계층 색인 데이터

In [26]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                  'key2': [2000, 2001, 2002, 2001, 2002],
                  'data': np.arange(5.)})

In [27]:
righth = DataFrame(np.arange(12).reshape((6,2)),
                  index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                        [2001, 2000, 2000, 2000, 2001, 2002]],
                  columns=['event1', 'event2'])

In [28]:
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [29]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [30]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [31]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000.0,4.0,5.0
0,0.0,Ohio,2000.0,6.0,7.0
1,1.0,Ohio,2001.0,8.0,9.0
2,2.0,Ohio,2002.0,10.0,11.0
3,3.0,Nevada,2001.0,0.0,1.0
4,4.0,Nevada,2002.0,NaN,NaN
4,NaN,Nevada,2000.0,2.0,3.0


In [32]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                 columns=['Ohio', 'Nevada'])

In [33]:
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                  index=['b', 'c', 'd', 'e'],
                  columns=['Missouri', 'Alabama'])

In [34]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [35]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [36]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [37]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [38]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


색인 대 색인으로 두 DataFrame 을 합치려면 그냥 간단히 머지하려는 DataFrame 의 리스트를 join 메서드에 넘기면 된다.

In [39]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                   index=['a', 'c', 'e', 'f'],
                   columns=['New York', 'Oregon'])

In [40]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [41]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 7.1.3. 축 따라 이어붙이기

In [42]:
arr = np.arange(12).reshape((3,4))

In [43]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [44]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

Series 나 DataFrame 같은 pandas 객체의 컨텍스트 내부에는 축마다 이름이 있어서 배열을 쉽게 이어 붙일 수 있도록 되어 있다.

* 만약에 연결하려는 두 객체의 색인이 서로 다르다면 결과는 그 색인의 교집합이어야 하는가? 아니면 합집합이어야 하는가?
* 합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할 수 있는가?
* 어떤 축으로 연결할 것인지 고려해야 하는가?

In [45]:
s1 = Series([0, 1], index=['a', 'b'])

In [46]:
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])

In [47]:
s3 = Series([5, 6], index=['f', 'g'])

In [48]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [49]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [50]:
s4 = pd.concat([s1*5, s3])

In [51]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [52]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [53]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


Series 를 이어붙이기 전의 개별 series 를 구분할 수 없다는 문제가 생기는데, 이 경우 계층적 색인을 생성하여 식별이 가능하도록 할 수 있다.

In [54]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])

In [55]:
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [56]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [57]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [58]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])

In [59]:
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])

In [60]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [61]:
pd.concat({'level1': df1, 'level2':df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [62]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

DataFrame의 로우 색인이 분석에 불필요한 경우에는 어떻게 할 것인가?<br/>
ignore_index=True 옵션을 이용하면 된다.

In [63]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [64]:
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd' ,'e'])

In [65]:
df1

,a,b,c,d
0,-1.279854,0.156097,-0.879818,-0.721974
1,-0.716373,-0.914999,0.048261,0.124199
2,1.283556,0.355190,0.954939,-0.234765


In [66]:
df2

,b,d,e
0,0.197202,-1.313105,1.721923
1,2.134651,-0.458554,-1.556882


In [67]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d,e
0,-1.279854,0.156097,-0.879818,-0.721974,NaN
1,-0.716373,-0.914999,0.048261,0.124199,NaN
2,1.283556,0.355190,0.954939,-0.234765,NaN
3,NaN,0.197202,NaN,-1.313105,1.721923
4,NaN,2.134651,NaN,-0.458554,-1.556882


concat 함수 인자

<table align="left">
    <thead>
        <td>인자</td>
        <td>설명</td>
    </thead>
    <tbody>
        <tr>
            <td>objs</td>
            <td>이어붙일 pandas 객체의 사전이나 리스트, 필수인자</td>
        </tr>
        <tr>
            <td>axis</td>
            <td>이어붙일 축 방향, 기본 값은 0</td>
        </tr>
        <tr>
            <td>join</td>
            <td>조인 방식, 'inner'(내부조인, 교집합, 기본값) 와 'outer'(외부조인, 합집합) 가 있음</td>
        </tr>
        <tr>
            <td>join_axes</td>
            <td>합집합/교집합을 수행하는 대신 다른 n-1 축으로 사용할 색인을 지정한다.</td>
        </tr>
        <tr>
            <td>keys</td>
            <td>이어붙일 객체나 이어붙인 축에 대한 계층 색인을  생성하는데 연관된 값이다. 리스트나 임의의 값이 들어있는 배열, 튜플의 배열 또는 배열의 리스트가 될 수 있다.</td>
        </tr>
        <tr>
            <td>levels</td>
            <td>계층 색인 레벨로 사용할 색인을 지정한다. keys 가 넘어온 경우 여러 개의 색인을 지정한다.</td>
        </tr>
        <tr>
            <td>names</td>
            <td>keys 나 levels 혹은 둘 다 있을 경우. 생성된 계층 레벨을 위한 이름</td>
        </tr>
        <tr>
            <td>verify_integrity</td>
            <td>이어붙인 객체에 중복되는 축이 있는지 검사하고 있다면 예외를 발생시킨다. 기본 값은 False로 중복을 허용한다</td>
        </tr>
        <tr>
            <td>ignore_index</td>
            <td>이어붙인 축의 색인을 유지하지 않고 range(total_length)로 새로운 색인을 생성한다</td>
        </tr>
    </tbody>
</table>

### 7.1.4. 겹치는 데이터 합치기

In [68]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
          index=['f', 'e', 'd', 'c', 'b', 'a'])

In [69]:
b = Series(np.arange(len(a), dtype=float), index=['f', 'e', 'd', 'c', 'b', 'a'])

In [70]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [71]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [72]:
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  5. ])

combine_first() 라는 메서드는 위와 같은 연산을 제공하며, 데이터 정렬 기능까지 제공한다.

In [73]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

DataFrame 에서 combine_first() 메서드는 칼럼에 대해 같은 동작을 한다. 그래서 이를 통해 호출하는 객체에서 누락된 데이터를 인자로 넘긴 깨체에 있는 값으로 채워 넣을 수 있다.

In [74]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                'b': [np.nan, 2., np.nan, 6.],
                'c': range(2, 18, 4)})

In [75]:
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                'b': [np.nan, 3., 4., 6., 8.]})

In [76]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 7.2. 재형성과 피벗

표 형식의 데이터를 재배치하는 다양한 기본 연산이 존재하는데, 이런 연산을 재형성 또는 피벗 연산이라고 한다.

### 7.2.1. 계층적 색인으로 재형성하기

* stack : 데이터의 칼럼을 로우로 피벗 또는 회전시킨다.
* unstack : 로우를 칼럼으로 피벗시킨다.

In [77]:
data = DataFrame(np.arange(6).reshape((2,3)),
                index=pd.Index(['Ohio', 'Colorado'], name='state'),
                columns=pd.Index(['one', 'two', 'three'], name='number'))

In [78]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [79]:
result = data.stack()

In [80]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [81]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [82]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [83]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [84]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [85]:
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])

In [86]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [87]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [88]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [89]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [90]:
df = DataFrame({'left': result, 'right': result + 5},
              columns=pd.Index(['left', 'right'], name='side'))

In [91]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [92]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [93]:
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

### 7.2.2. 피벗팅으로 데이터 나열 방식 바꾸기

In [94]:
data = pd.read_csv('ch07/macrodata.csv')
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
data = DataFrame(data.to_records(),
                columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                index=periods.to_timestamp('D', 'end'))
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [95]:
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [96]:
# pivot (row, column, value)
pivoted = ldata.pivot('date', 'item', 'value')

In [97]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [98]:
ldata['value2'] = np.random.randn(len(ldata))

In [99]:
ldata[:10]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,1.308034
1,1959-03-31,infl,0.000,-0.201751
2,1959-03-31,unemp,5.800,0.813335
3,1959-06-30,realgdp,2778.801,0.790450
4,1959-06-30,infl,2.340,-0.641490
5,1959-06-30,unemp,5.100,0.395111
6,1959-09-30,realgdp,2775.488,1.206959
7,1959-09-30,infl,2.740,1.189230
8,1959-09-30,unemp,5.300,1.272244
9,1959-12-31,realgdp,2785.204,0.134742


In [100]:
pivoted = ldata.pivot('date', 'item')

In [101]:
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.201751  1.308034  0.813335
1959-06-30  2.34  2778.801   5.1 -0.641490  0.790450  0.395111
1959-09-30  2.74  2775.488   5.3  1.189230  1.206959  1.272244
1959-12-31  0.27  2785.204   5.6  1.221230  0.134742 -0.640250
1960-03-31  2.31  2847.699   5.2  0.499820  0.603166  0.460062

In [102]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [103]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

In [104]:
unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.201751  1.308034  0.813335
1959-06-30  2.34  2778.801   5.1 -0.641490  0.790450  0.395111
1959-09-30  2.74  2775.488   5.3  1.189230  1.206959  1.272244
1959-12-31  0.27  2785.204   5.6  1.221230  0.134742 -0.640250
1960-03-31  2.31  2847.699   5.2  0.499820  0.603166  0.460062
1960-06-30  0.14  2834.390   5.2  0.987870 -1.103822 -0.275901
1960-09-30  2.70  2839.022   5.6  0.108421  1.048735  1.239982

## 7.3. 데이터 변형

### 7.3.1. 중복 제거하기

In [105]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                 'k2': [1, 1, 2, 3, 3, 4, 4]})

In [106]:
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [107]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [108]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [109]:
data['v1']=range(7)

In [110]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [111]:
# keep = (first(기본값), last)
data.drop_duplicates(['k1','k2'],keep='last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


### 7.3.2. 함수나 매핑 이용해 데이터 변형하기

In [112]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 
                           'Pastrami', 'corned beef', 'Bacon', 
                           'pastrami', 'honey ham', 'nova lox'],
                 'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [113]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [114]:
meat_to_animal = {
    'bacon':'pig',
    'pulled pork':'pig',
    'pastrami':'cow',
    'corned beef':'cow',
    'honey ham':'pig',
    'nova lox':'salmon'
}

In [115]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)

In [116]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [117]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 7.3.3. 값 치환하기

fillna 메서드를 사용해서 누락된 값을 채우는 일은 일반적인 값 치환작업이다.

In [118]:
data = Series([1., -999., 2., -999., -1000., 3.])

In [119]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [120]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [121]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [122]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [123]:
data.replace({-999:np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.3.4. 축 색인 이름 바꾸기

In [124]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                index=['Ohio', 'Colorado', 'New York'],
                columns=['one', 'two', 'three', 'four'])

In [125]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [126]:
data.index.map(str.upper)

array(['OHIO', 'COLORADO', 'NEW YORK'], dtype=object)

In [127]:
data.index = data.index.map(str.upper)

In [128]:
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [129]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [130]:
data.rename(index={'Ohio':'INDIANA'},
           columns={'three': 'peekaboo'})

,one,two,peekaboo,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [131]:
# 항상 DataFrame 의 참조를 반환한다
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [132]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### 7.3.5. 개별화와 양자화

연속성 데이터는 종종 개별로 분할하거나 분석을 위해 그룹별로 나누기도 한다.<br/>
예를 들어 연령 같은 정보를 들 수 있다. 10대, 20대, 30대 등등

In [133]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [134]:
bins = [18, 25, 35, 60, 100]

In [135]:
cats = pd.cut(ages, bins)

In [136]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

Categorical 객체란?<br/>
그룹 이름이 담긴 배열.
Categorical 객체는 codes 속성에 있는 ages 데이터에 대한 카티고리 이름을 categories 라는 배열에 내부적으로 담고 있다.

In [137]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [138]:
cats.categories

Index(['(18, 25]', '(25, 35]', '(35, 60]', '(60, 100]'], dtype='object')

In [139]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

간격을 나타내는 표기법은 괄호를 시작해서 대괄호로 끝나는데, 괄호 쪽의 값은 포함하지 않고 대괄호 쪽의 값은 포함하는 간격을 나타낸다.<br/>
right = False 를 넘겨서 괄호와 대괄호의 위치를 바꿀 수 있다.

In [140]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, object): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [141]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

카테고리 이름을 별도로 지정해 줄 수 도 있다.

In [142]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

cut 함수는 명시적으로 그룹의 경계 값을 넘기지 않고 그룹의 개수를 넘겨주면 데이터에서 최소 값과 최대 값을 기준으로 균등한 길이의 그룹을 자동으로 계산한다.

In [143]:
# 균등분포로 그룹을 나눠보자.
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.26, 0.51], (0.51, 0.75], (0.022, 0.26], (0.022, 0.26], (0.022, 0.26], ..., (0.26, 0.51], (0.26, 0.51], (0.26, 0.51], (0.51, 0.75], (0.75, 0.99]]
Length: 20
Categories (4, object): [(0.022, 0.26] < (0.26, 0.51] < (0.51, 0.75] < (0.75, 0.99]]

In [144]:
data = np.random.randn(1000)

In [145]:
cats = pd.qcut(data, 4)
cats

[(0.0599, 0.722], (0.722, 3.033], (0.722, 3.033], [-3.397, -0.713], (0.0599, 0.722], ..., (0.0599, 0.722], (0.722, 3.033], (-0.713, 0.0599], (0.722, 3.033], (0.722, 3.033]]
Length: 1000
Categories (4, object): [[-3.397, -0.713] < (-0.713, 0.0599] < (0.0599, 0.722] < (0.722, 3.033]]

cut 함수와 유사하게 변위치를 직접 지정해 줄 수 있다.(변위치 값은 0부터 1까지다)

In [146]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(0.0599, 1.294], (0.0599, 1.294], (1.294, 3.033], [-3.397, -1.293], (0.0599, 1.294], ..., (0.0599, 1.294], (0.0599, 1.294], (-1.293, 0.0599], (0.0599, 1.294], (1.294, 3.033]]
Length: 1000
Categories (4, object): [[-3.397, -1.293] < (-1.293, 0.0599] < (0.0599, 1.294] < (1.294, 3.033]]

### 7.3.6. 특이값 찾아내고 제외하기

In [147]:
np.random.seed(12345)

In [148]:
data = DataFrame(np.random.randn(1000, 4))

In [149]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


DataFrame 의 한 칼럼에서 절대 값이 3을 초과하는 값을 찾아내보자.

In [150]:
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [151]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [152]:
data[np.abs(data) > 3] = np.sign(data)*3

In [153]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### 7.3.7. 치환과 임의 샘플링

num.random.permutation 함수를 이용하면 Series 나 DataFrame 의 로우를 쉽게 임의의 순서대로 재배치할 수 있다.

In [154]:
df = DataFrame(np.arange(5*4).reshape(5, 4))

In [155]:
sampler = np.random.permutation(5)

In [156]:
sampler

array([1, 0, 2, 3, 4])

In [157]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [158]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


치환없이 일부만 임의로 선택하려면?

In [159]:
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


치환을 통해 표본을 생성하기 위한 가장 빠른 방법은 np.random.randint를 사용해서 성성한 난수를 이용하는 방법이다.

In [160]:
bag = np.array([5, 7, -1, 6, 4])

In [161]:
sampler = np.random.randint(0, len(bag), size=10)

In [162]:
sampler

array([4, 4, 2, 2, 2, 0, 3, 0, 4, 1])

In [163]:
draws = bag.take(sampler)

In [164]:
draws

array([ 4,  4, -1, -1, -1,  5,  6,  5,  4,  7])

### 7.3.8. 표시자/더미 변수

In [165]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
               'data1':range(6)})

In [166]:
pd.get_dummies(df['key'])

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


표시용 DataFrame 안에 있는 칼럼에 접두어를 추가한 후 다른 데이터와 병합하고 싶을 때가 있는데, 이런 경우 get_dummies 함수의 prefix 인자를 사용하면 된다.

In [167]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [168]:
df_with_dummy = df[['data1']].join(dummies)

In [169]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


DataFrame 의 한 칼럼이 여러 카테고리에 속한다면?

In [170]:
names = ['movie_id', 'title', 'genres']

In [171]:
movies = pd.read_table('ch02/movielens/movies.dat', sep='::', header=None, names=names)

/Users/NAVER/Documents/dev/dev_env/Anaconda/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [172]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [173]:
# 각 장르 목록 추출
genre_iter = (set(x.split('|')) for x in movies.genres)

In [174]:
# 추출된 장르를 set.union 을 이용하여 유일한 값으로 추출
genres = sorted(set.union(*genre_iter))

In [175]:
# 표시용 DataFrame 을 생성하여 0으로 초기화
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)

In [176]:
# movies.genres 데이터를 dummies 데이터로 생성
for i, gen in enumerate(movies.genres):
    dummies.ix[i, gen.split('|')] = 1

In [177]:
# 실제 movies 데이터와 join
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [178]:
movies_windic.ix[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

get_dummies 나 cut 같은 이산함수를 잘 활용하면 통계 애플리케이션에서 유용하게 사용할 수 있다.

In [179]:
values = np.random.rand(10)

In [180]:
values

array([ 0.75603383,  0.90830844,  0.96588737,  0.17373658,  0.87592824,
        0.75415641,  0.163486  ,  0.23784062,  0.85564381,  0.58743194])

In [181]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [182]:
pd.get_dummies(pd.cut(values, bins))

,"(0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1]"
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,1.0,0.0
6,1.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,1.0,0.0,0.0


## 7.4. 문자열 다루기

* 정규표현식
* 누락된 데이터 처리(정규표현식 활용)

### 7.4.1. 문자열 객체 메서드

In [183]:
val = 'a,b,   guido'

In [184]:
val.split(',')

['a', 'b', '   guido']

In [185]:
pieces = [x.strip() for x in val.split(',')]

In [186]:
pieces

['a', 'b', 'guido']

In [187]:
first, second, third = pieces

In [188]:
first + '::' + second + '::' + third

'a::b::guido'

In [189]:
'::'.join(pieces)

'a::b::guido'

In [190]:
'guido' in val

True

In [191]:
val.index(',')

1

In [192]:
val.find(':')

-1

index 의 경우 찾고자 하는 문자열을 발견하지 못하면 Exception 이 발생한다.

In [193]:
val.index('::')

ValueError: substring not found

In [194]:
val.count(',')

2

In [195]:
val.replace(',', '::')

'a::b::   guido'

In [196]:
val.replace(',', '')

'ab   guido'

파이썬 내장 문자열 메서드

<table align="left">
    <thead>
        <td>인자</td>
        <td>설명</td>
    </thead>
    <tbody>
        <tr>
            <td>count</td>
            <td>문자열에서 겹치지 않는 부분 문자열의 개수를 반환한다.</td>
        </tr>
        <tr>
            <td>endswith, startswith</td>
            <td>문자열이 주어진 접미사(점두사)로 끝날 경우 True 를 반환한다.</td>
        </tr>
        <tr>
            <td>join</td>
            <td>문자열을 구분자로 하여 다른 문자열을 순서대로 이어붙인다.</td>
        </tr>
        <tr>
            <td>index</td>
            <td>부분 문자열의 첫 번째 글자의 위치를 반환한다. 부분 문자열이 없다면 ValueError 예외가 발생한다.</td>
        </tr>
        <tr>
            <td>find</td>
            <td>첫 번째 부분 문자열의 첫 번째 글자의 위치를 반환한다. index 와 유사하지만 부분 문자열이 없을 경우 -1을 반환한다.</td>
        </tr>
        <tr>
            <td>rfind</td>
            <td>마지막 부분 문자열의 첫 번째 글자의 위치를 반환한다. 부분 문자열이 없다면 -1을 반환한다.</td>
        </tr>
        <tr>
            <td>replace</td>
            <td>문자열을 다른 문자열로 치환한다.</td>
        </tr>
        <tr>
            <td>strip, rstrip<br/>lstrip</td>
            <td>개행문자를 포함한 공백문자를 제거한다. lstrip은 문자열의 시작 부분에 있는 공백문자만을 제거하고, rstrip은 문자열의 마지막 부분에 있는 공백문자만을 제거한다.</td>
        </tr>
        <tr>
            <td>split</td>
            <td>문자열을 구분자를 기준으로 부분 문자열의 리스트로 분리한다.</td>
        </tr>
        <tr>
            <td>lower, upper</td>
            <td>알파벳 문자를 소문자 혹은 대문자로 변환한다.</td>
        </tr>
        <tr>
            <td>ljust, rjsut</td>
            <td>문자열을 오른쪽 혹은 왼쪽으로 정렬하고 주어진 길이에서 문자열의 길이를 제외한 곳은 공백문자를 채워 넣어 주어진 길이를 가지는 문자열을 반환한다.</td>
        </tr>
    </tbody>
</table>

### 7.4.2. 정규표현식

In [197]:
import re

In [198]:
text = "foo    bar\t baz   \tqux"

In [199]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [200]:
regex = re.compile('\s+')

In [201]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

정규표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 을 사용하자

In [202]:
regex.findall(text)

['    ', '\t ', '   \t']

In [204]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+=]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE 는 정규표현식이 대/소문자를 가리지 않도록 한다.
regex = re.compile(pattern, flags=re.IGNORECASE)

In [205]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [206]:
m = regex.search(text)

In [207]:
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [208]:
text[m.start():m.end()]

'dave@google.com'

In [209]:
print(regex.match(text))

None


In [210]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



이메일 주소를 찾아서 동시에 각 이메일 주소를 사용자 이름, 도메인 이름, 도메인 접미사의 세가지 컴포넌트로 나누어야 한다면 각 패턴을 괄호로 묶어준다.

In [211]:
pattern = r'([A-Z0-9._%+=]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [212]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [213]:
m = regex.match('web@bright.net')

In [214]:
m.groups()

('web', 'bright', 'net')

In [215]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [216]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [225]:
regex = re.compile(r'''(?P<username>[A-Z0-9._%+=]+)@(?P<domain>[A-Z0-9.-]+)\.(?P<suffix>[A-Z]{2,4})''', 
                   flags=re.IGNORECASE|re.VERBOSE)

In [226]:
m = regex.match('web@bright.net')

In [227]:
m.groupdict()

{'domain': 'bright', 'suffix': 'net', 'username': 'web'}

정규표현식 메서드

<table align="left">
    <thead>
        <td>인자</td>
        <td>설명</td>
    </thead>
    <tbody>
        <tr>
            <td>findall, finditer</td>
            <td>문자열에서 겹치지 않는, 발견된 모든 패턴을 반환한다. findall 메서드는 모든 패턴의 리스트를 반환하고, finditer 메서드는 이터레이터를 통해 하나씩 반환한다.</td>
        </tr>
        <tr>
            <td>match</td>
            <td>문자열의 시작점부터 패턴을 찾고 선택적으로 패턴 컴포넌트를 그룹으로 나눈다. 일치하는 패턴이 있다며 match 객체를 반환하고 그렇지 않은 경우 None 을 반환한다.</td>
        </tr>
        <tr>
            <td>search</td>
            <td>문자열에서 패턴과 일치하는 내용을 검색ㅎ match 객체를 반환한다. match 메서드와는 달리 시작부터 일치하는 내용만 찾지 않고 문자열 어디든 일치하는 내용이 있다면 반환한다.</td>
        </tr>
        <tr>
            <td>split</td>
            <td>문자열에서 패턴과 일치하는 부분을 쪼갠다.</td>
        </tr>
        <tr>
            <td>sub, subn</td>
            <td>aㅜㄴ자열에서 일치하는 모든 패턴(sub) 혹은 처음 n개의 패턴(subn)을 대체 표현으로 치환한다. 대체 표현 문자열은 \1, \2, ... 같은 기호를 사용해서 매치 그룹의 요소를 참조 한다.</td>
        </tr>
    </tbody>
</table>

### 7.4.3. pandas 벡터화된 문자열 상수

In [228]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
       'Rob': 'rob@gmail.com', 'Wes':np.nan}

In [229]:
data = Series(data)

In [230]:
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [232]:
data.isnull()

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

In [233]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [234]:
pattern

'([A-Z0-9._%+=]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [235]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

In [236]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

/Users/NAVER/Documents/dev/dev_env/Anaconda/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.
  if __name__ == '__main__':


In [237]:
matches

Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

In [238]:
matches.str.get(1)

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

In [239]:
matches.str[0]

Dave      dave
Rob        rob
Steve    steve
Wes        NaN
dtype: object

In [240]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

벡터화된 문자열 메서드

<table align="left">
    <thead>
        <td>메서드</td>
        <td>설명</td>
    </thead>
    <tbody>
        <tr>
            <td>cat</td>
            <td>선택적인 구분자와 함께 요소별로 문자열을 이어붙인다.</td>
        </tr>
        <tr>
            <td>contains</td>
            <td>문자열이 패턴이나 정규표현식을 포함하는지를 나타내는 불리언 배열을 반환한다.</td>
        </tr>
        <tr>
            <td>count</td>
            <td>일치하는 패턴의 개수를 반환한다.</td>
        </tr>
        <tr>
            <td>endswith, startswith</td>
            <td>각 요소에 대해 x.endswith(pattern), x.startswith(pattern) 와 동일한 동작을 한다.</td>
        </tr>
        <tr>
            <td>findall</td>
            <td>각 문자열에 대해 일치하는 패턴/정규표현식의 전체 목록을 구한다.</td>
        </tr>
        <tr>
            <td>get</td>
            <td>i 번째 요소를 반환한다.</td>
        </tr>
        <tr>
            <td>join</td>
            <td>Series 의 각 요소를 주어진 구분자로 연결한다.</td>
        </tr>
        <tr>
            <td>len</td>
            <td>각 문자열의 길이를 구한다.</td>
        </tr>
        <tr>
            <td>lower, upper</td>
            <td>대/소문자로 변환한다. 각 요소에 대한 x.lower(), x.upper() 와 같다.</td>
        </tr>
        <tr>
            <td>match</td>
            <td>주어진 정규표현식으로 각 요소에 대한 re.match를 수행하여 일치하는 그룹을 리스트로 반환한다.</td>
        </tr>
        <tr>
            <td>pad</td>
            <td>문자열의 좌, 우 혹은 양쪽에 공백을 추가한다.</td>
        </tr>
        <tr>
            <td>center</td>
            <td>pad(side='both')와 동일하다.</td>
        </tr>
        <tr>
            <td>repeat</td>
            <td>값을 복사한다. 예를 들어 s.str.repeat(3)는 각 문자열에 대한 x*3과 동일하다.</td>
        </tr>
        <tr>
            <td>replace</td>
            <td>패턴/정규표현식과 일치하는 내용을 다른 문자열로 치환한다.</td>
        </tr>
        <tr>
            <td>slice</td>
            <td>Series 안에 있는 각 문자열을 자른다.</td>
        </tr>
        <tr>
            <td>split</td>
            <td>정규표현식 혹은 구분자로 문자열을 나눈다.</td>
        </tr>
        <tr>
            <td>strip, rstrip, lstrip</td>
            <td>각 요소의 개행문자를 포함한 공백문자를 제거한다</td>
        </tr>
    </tbody>
</table>